In [2]:
#OPEN_CV AND PIL IMPORTS
import numpy as np
from PIL import ImageGrab
from PIL import Image
import cv2
import io
import time

In [3]:
#LIBRARY IMPORTS
%matplotlib inline 
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (15, 9)
import seaborn as sns
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint

In [4]:
#SELENIUM IMPORTS
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [5]:
#KERAS IMPORTS
%matplotlib inline 
from keras.models import model_from_json
from keras.models import Sequential
from keras.models import load_model

from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
import json

Using TensorFlow backend.


In [6]:
#PATH VARIABLES
game_url = "game/dino.html"
chrome_driver_path = "./chromedriver.exe"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
scores_file_path = "./objects/scores_df.csv"

In [7]:
#GAME MODULE
'''
* Game class: Selenium interfacing between the python and browser
* __init__():  Launch the broswer window using the attributes in chrome_options
* crashed() : return true if the agent as crashed on an obstacles. Gets javascript variable from game decribing the state
* playing(): true if game in progress, false is crashed or paused
* restart() : sends a signal to browser-javascript to restart the game
* press_up(): sends a single to press up get to the browser
* get_score(): gets current game score from javascript variables.
* pause(): pause the game
* resume(): resume a paused game if not crashed
* end(): close the browser and end the game
'''
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(200, 300)
        self._driver.get(os.path.abspath(game_url))
        #modifying game before trainNetworkining
        if custom_config:
            self._driver.execute_script("Runner.config.ACCELERATION=0")
    def crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        self._driver.execute_script("Runner.instance_.restart()")
        
        time.sleep(0.25)# no actions are possible 
                        # for 0.25 sec after game starts, 
                        # skip learning at this time and make the model wait
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array) # the javascript object is of type array with score in the formate[1,0,0] which is 100.
        return int(score)
    def pause(self):
        return self._driver.execute_script("return Runner.instance_.stop()")
    def resume(self):
        return self._driver.execute_script("return Runner.instance_.play()")
    def end(self):
        self._driver.close()

In [8]:
#AGENT MODULE
class Agent_Dino:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game; 
        self.jump(); #to start the game, we need to jump once
        time.sleep(.5) # no action can be performed for the first time when game starts
    def is_running(self):
        return self._game.playing()
    def is_crashed(self):
        return self._game.crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

In [9]:
#GAME STATE MODULE
class Game_state:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = display_images() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        score = self._game.get_score() 
        reward = 0.1*score/10 # dynamic reward calculation
        is_over = False #game over
        if actions[1] == 1:
            self._agent.jump()
            reward = 0.1*score/11
        image = screen_capture() 
        self._display.send(image) #display the image on screen

        if self._agent.is_crashed():
            scores_df.loc[len(loss_df)] = score # log the score when game is over
            self._game.restart()
            reward = -11/score
            is_over = True
        return image, reward, is_over #return the Experience tuple

In [10]:
def save(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load(name):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
def screen_capture(_driver = None):
    screen =  np.array(ImageGrab.grab(bbox=(40,180,440,400))) #bbox = region of interset on the entire screen
    image = image_processing(screen)#processing image as required
    return image

def image_processing(image):
    #game is already in grey scale canvas, canny to get only edges and reduce unwanted objects(clouds)
    image = cv2.resize(image, (0,0), fx = 0.15, fy = 0.10) # resale image dimensions
    image = image[2:38,10:50] #img[y:y+h, x:x+w] #crop out the dino agent from the frame
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200) #apply the canny edge detection
    return  image
def display_images(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break


In [11]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])

In [12]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save(INITIAL_EPSILON,"epsilon")
    t = 0
    save(t,"time")
    D = deque()
    save(D,"D")

In [13]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 50000. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
image_rows , image_columns = 40,20
image_channels = 4 #We stack 4 frames

In [14]:
#MODEL BUILDING
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same',input_shape=(image_columns,image_rows,image_channels)))  #20*40*4
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

In [15]:
#MODEL ARCHITECTURE
buildmodel().summary()

Now we build the model
Instructions for updating:
Colocations handled automatically by placer.
We finish building the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 5, 10, 32)         8224      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 10, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 5, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 3, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 5, 64)          36928     
_________________________________________________________________
activation_3 (Activation)    (None, 3, 5, 64)          0         
__________________

In [16]:
#MAIN TRAINING MODULE
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon1 = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model_final.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load("epsilon") 
        model.load_weights("model_final.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[0] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('loop took {} seconds'.format(time.time()-last_time)) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
        else:
            # artificial time delay as training done with this delay
            time.sleep(0.12)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            
            model.save_weights("model_final.h5", overwrite=True)
            save(D,"D") #saving episodes
            save(t,"time") #caching time steps
            save(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [17]:
def playGame(observe=False):
    game = Game()
    dino = Agent_Dino(game)
    game_state = Game_state(dino,game)
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [18]:
init_cache()
playGame(observe=False);

C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


Now we build the model
We finish building the model
----------Random Action----------
loop took 0.8432583808898926 seconds
TIMESTEP 1 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.07 / Q_MAX  0 / Loss  0
loop took 0.5654876232147217 seconds
TIMESTEP 2 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.27227163314819336 seconds
TIMESTEP 3 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1272727272727273 / Q_MAX  0 / Loss  0
loop took 0.24982857704162598 seconds
TIMESTEP 4 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.14545454545454548 / Q_MAX  0 / Loss  0
loop took 0.25930237770080566 seconds
TIMESTEP 5 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.17272727272727273 / Q_MAX  0 / Loss  0
loop took 0.2822229862213135 seconds
TIMESTEP 6 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.19090909090909092 / Q_MAX  0 / Loss  0
loop took 0.2812471389770508 seconds
TIMESTEP 7 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.209090909

loop took 0.3311150074005127 seconds
TIMESTEP 59 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.13636363636363635 / Q_MAX  0 / Loss  0
loop took 0.33330702781677246 seconds
TIMESTEP 60 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.16363636363636364 / Q_MAX  0 / Loss  0
loop took 0.31595730781555176 seconds
TIMESTEP 61 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.19090909090909092 / Q_MAX  0 / Loss  0
loop took 0.354053258895874 seconds
TIMESTEP 62 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.21818181818181823 / Q_MAX  0 / Loss  0
----------Random Action----------
loop took 0.26928138732910156 seconds
TIMESTEP 63 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.27 / Q_MAX  0 / Loss  0
loop took 0.2582082748413086 seconds
TIMESTEP 64 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.26363636363636367 / Q_MAX  0 / Loss  0
loop took 0.28324270248413086 seconds
TIMESTEP 65 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.2818181818181818 / Q_MAX  0 / Loss  0

loop took 0.42985105514526367 seconds
TIMESTEP 117 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.19090909090909092 / Q_MAX  0 / Loss  0


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)
